## <p style="color:red;text-align:center">**News Classifier**</p>

### <p style="text-align:center">**ML Pipeline with Sentence Transformers, ChromaDB & Airflow**</p>

Ce projet vise à construire un pipeline complet de classification de textes permettant de catégoriser automatiquement des articles d’actualité dans 4 classes :

- World
- Sports
- Business
- Sci/Tech

Le pipeline utilise des techniques NLP avancées, des embeddings vectoriels, une base de données vectorielle et une orchestration via Apache Airflow.

<br>

### <span style="color:green">**Entraînement & Évaluation :**</span>

#### <span style="color:orange">**1. Charger les Données :**</span>

##### **1.1. Etablir la Connexion avec ChromaDB :**

In [1]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="../chroma_db")

collections = client.list_collections()

print("Collections Disponibles :")

for col in collections:
    print(f"- {col.name}")

Collections Disponibles :
- test_news
- train_news


##### **1.2. Récupèrer les Collections :**

In [2]:
train_collection = client.get_collection(name=collections[1].name)
test_collection = client.get_collection(name=collections[0].name)

train_data = train_collection.get(include=['embeddings','metadatas'])
test_data = test_collection.get(include=['embeddings','metadatas'])

#### <span style="color:orange">**2. Séparer `X_train`, `y_train`, `X_test`, `y_test` :**</span>

In [3]:
import numpy as np

X_train = np.array(train_data['embeddings'])
X_test = np.array(test_data['embeddings'])

y_train = np.array([data['label'] for data in train_data['metadatas']])
y_test = np.array([data['label'] for data in test_data['metadatas']])

print(f"- Dimensions X_train : {X_train.shape}")
print(f"- Dimensions y_train : {y_train.shape}")
print(f"- Dimensions X_test  : {X_test.shape}")
print(f"- Dimensions y_test  : {y_test.shape}")

- Dimensions X_train : (120000, 384)
- Dimensions y_train : (120000,)
- Dimensions X_test  : (7600, 384)
- Dimensions y_test  : (7600,)


In [4]:
print("\n- 5 Premiers Labels Train :", y_train[:5])

print("\n- 5 Premiers Labels Test :", y_test[:5])


- 5 Premiers Labels Train : [2 2 2 2 2]

- 5 Premiers Labels Test : [2 3 3 3 3]
